In [ ]:

import pandas as pd
import random
import time
import socket

INPUT_CSV = "./group212252217_http_input.csv" 

df = pd.read_csv(INPUT_CSV)
print("Loaded CSV:")
df.head()

In [ ]:
def simulate_encapsulation(row, src_ip="192.168.1.10", dst_ip="93.184.216.34"):
  
    msg = row["message"]
    
    # שכבת יישום
    application_layer = {
        "protocol": row["app_protocol"],
        "payload": msg
    }
    
    # שכבת תעבורה – TCP (ערכים מומצאים)
    transport_layer = {
        "protocol": "TCP",
        "src_port": random.randint(1024, 65535),
        "dst_port": 80,
        "seq_num": random.randint(1000, 5000),
        "ack_num": random.randint(5000, 9000),
        "flags": "PSH, ACK"
    }
    
    # שכבת רשת – IP
    network_layer = {
        "protocol": "IPv4",
        "src_ip": src_ip,
        "dst_ip": dst_ip,
        "ttl": 64
    }
    
    # שכבת קו – Link (Ethernet)
    link_layer = {
        "src_mac": "AA:BB:CC:DD:EE:01",
        "dst_mac": "AA:BB:CC:DD:EE:02",
        "type": "Ethernet II"
    }
    
    packet = {
        "msg_id": row["msg_id"],
        "timestamp": row["timestamp"],
        "application": application_layer,
        "transport": transport_layer,
        "network": network_layer,
        "link": link_layer
    }
    return packet

packets = [simulate_encapsulation(row) for _, row in df.iterrows()]
len(packets), packets[0]


In [ ]:
# המרת המבנה ל-DataFrame שטוח לצורך תצוגה
records = []
for p in packets:
    rec = {
        "msg_id": p["msg_id"],
        "timestamp": p["timestamp"],
        "app_protocol": p["application"]["protocol"],
        "app_payload": p["application"]["payload"],
        "tcp_src_port": p["transport"]["src_port"],
        "tcp_dst_port": p["transport"]["dst_port"],
        "tcp_flags": p["transport"]["flags"],
        "ip_src": p["network"]["src_ip"],
        "ip_dst": p["network"]["dst_ip"],
        "link_src_mac": p["link"]["src_mac"],
        "link_dst_mac": p["link"]["dst_mac"],
    }
    records.append(rec)

packets_df = pd.DataFrame.from_records(records)
packets_df


In [ ]:
HOST = "0.0.0.0"  
PORT = 80

def send_http_traffic(df, host=HOST, port=PORT, delay=0.3):
  
    for _, row in df.iterrows():
        message = row["message"]
        
        if "HTTP/1.1" in message and "Host:" not in message:
            http_request = (
                f"{message}\r\n"
                f"Host: {host}\r\n"
                f"Connection: close\r\n"
                f"\r\n"
            )
        else:
            http_request = message + "\r\n\r\n"
        
        try:
            s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            s.connect((host, port))
            s.sendall(http_request.encode("utf-8"))
            _ = s.recv(1024)
        except Exception as e:
            print(f"Error sending message {row['msg_id']}: {e}")
        finally:
            s.close()
        
        time.sleep(delay)

print("Ready. Run send_http_traffic(df) after capturing Wireshark.")
